In [1]:
import os
import datetime

import numpy as np
import scipy
import fiona
import statistics
import math

import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import LineString, shape, mapping, Point, Polygon, MultiPolygon
from shapely.ops import cascaded_union

from scipy.stats import linregress, norm

import matplotlib.pyplot as plt
from matplotlib import colors, cm, style
import matplotlib.patches as mpatches
from descartes import PolygonPatch
from matplotlib.colors import LinearSegmentedColormap

import osmnx as ox
import networkx as nx

import pysal as ps
import libpysal as lps
import esda
import seaborn as sns

from tabulate import tabulate
from termcolor import colored

/Users/roosteeuwen/opt/anaconda3/envs/greenspaces/lib/python3.7/site-packages/pysal/__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://migrating.pysal.org
  ), VisibleDeprecationWarning)


In [2]:
style.use('seaborn-white')
input_output_folder = 'data'

# Parameters

In [3]:
local_crs = 'EPSG:28992'
sub_folder_1 = 'Amsterdam_14Apr2022'
sub_folder_2 = 'Rotterdam_14Apr2022'
sub_folder_3 = 'TheHague_14Apr2022'

In [4]:
trip_distances = [300, 500, 800]

# Load data

In [5]:
gs_1 = gpd.read_file(os.path.join(input_output_folder, sub_folder_1, 'ALTERNATIVE_greenspace_accessibility.geojson'))
gs_1['city'] = sub_folder_1.split('_')[0]
gs_2 = gpd.read_file(os.path.join(input_output_folder, sub_folder_2, 'ALTERNATIVE_greenspace_accessibility.geojson'))
gs_2['city'] = sub_folder_2.split('_')[0]
gs_3 = gpd.read_file(os.path.join(input_output_folder, sub_folder_3, 'ALTERNATIVE_greenspace_accessibility.geojson'))
gs_3['city'] = sub_folder_3.split('_')[0]

In [6]:
gs_list = [gs_1, gs_2, gs_3]
greenspaces = gpd.GeoDataFrame(pd.concat(gs_list, ignore_index=True))
greenspaces.crs = local_crs

In [7]:
def set_types_geometries(df, cols):
    for col in cols:
        df[col] = df[col].apply(wkt.loads)
        gdf = gpd.GeoDataFrame(df, geometry=col)
        gdf.crs = local_crs

In [8]:
cols = []
for trip_dist in trip_distances:
    cols.append('geom_iso_'+str(trip_dist))

set_types_geometries(greenspaces, cols)

In [9]:
for col in greenspaces.columns:
    if not 'geom' in col:
        if not 'city' in col:
            greenspaces[col] = greenspaces[col].astype(float)

# Descriptive statistics

In [10]:
cols = [
    'betw_adl_800_dr12_entering', 
    'betw_adl_1000_dr12_entering', 
    'betw_adl_1200_dr12_entering'
    ]

In [11]:
headers_descr = ['', 'N', 'mean', 'std.dev.', 'min', 'max']
rows_descr = []

for col in cols:
    row = [col]
    
    row.append(len(greenspaces[greenspaces[col].notna()]))
    row.append(round(greenspaces[col].mean(), 1))
    row.append(round(greenspaces[col].std(), 1))
    row.append(round(greenspaces[col].min(), 0))
    row.append(round(greenspaces[col].max(), 0))

    rows_descr.append(row)

In [12]:
print(tabulate(rows_descr, headers=headers_descr))

                               N    mean    std.dev.    min    max
---------------------------  ---  ------  ----------  -----  -----
betw_adl_800_dr12_entering   848    34.6       150.8      0   2287
betw_adl_1000_dr12_entering  848    51.5       174.8      0   1738
betw_adl_1200_dr12_entering  848    58.8       190.5      0   1859


In [13]:
len(greenspaces[greenspaces['betw_adl_800_dr12_wsum']>0.0])

238

In [14]:
len(greenspaces[greenspaces['betw_adl_1000_dr12_wsum']>0.0])

330

In [15]:
len(greenspaces[greenspaces['betw_adl_1200_dr12_wsum']>0.0])

371

# Correlations between accessibility measures

In [16]:
headers = ['']
rows = []

for col_a in cols:
    row = [col_a]
    
    for col_b in cols:
        if col_a == col_b:
            row.append(1)
            break
        else:
            r, p = scipy.stats.spearmanr(greenspaces[col_a], greenspaces[col_b])
            r = round(r, 3)
            
            if p <= 0.05:
                r = str(r)+'*'
                if p <= 0.01:
                    r = r+'*'

            row.append(r)
    headers.append(col_a)
    rows.append(row)

In [17]:
print(tabulate(rows, headers=headers))

                             betw_adl_800_dr12_entering    betw_adl_1000_dr12_entering      betw_adl_1200_dr12_entering
---------------------------  ----------------------------  -----------------------------  -----------------------------
betw_adl_800_dr12_entering   1
betw_adl_1000_dr12_entering  0.867**                       1
betw_adl_1200_dr12_entering  0.753**                       0.841**                                                    1
